# End-to-End Retail Price Optimization Engine

**Goal:** Predict total retail price and surface top pricing drivers for interpretability.  
**Pipeline:** load → preprocess → feature selection (ANOVA F-test) → model training (Gradient Boosting) → evaluation → exports for Power BI.  


In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import mean_absolute_error, r2_score

# Load the dataset
# Ensure 'retail_price.csv' is in the same directory or provide the full path
try:
    data = pd.read_csv("C:/Res_Projects/Price_Optimization_Engine/retail_price.xls")
    print("Dataset loaded successfully.")
    print("Data Head:")
    print(data.head())
except FileNotFoundError:
    print("Error: 'retail_price.csv' not found. Please download it and place it in the correct directory.")
    # Create a dummy dataframe to prevent further errors if the file is missing
    data = pd.DataFrame()


Dataset loaded successfully.
Data Head:
  product_id product_category_name  month_year  qty  total_price  \
0       bed1        bed_bath_table  01-05-2017    1        45.95   
1       bed1        bed_bath_table  01-06-2017    3       137.85   
2       bed1        bed_bath_table  01-07-2017    6       275.70   
3       bed1        bed_bath_table  01-08-2017    4       183.80   
4       bed1        bed_bath_table  01-09-2017    2        91.90   

   freight_price  unit_price  product_name_lenght  product_description_lenght  \
0      15.100000       45.95                   39                         161   
1      12.933333       45.95                   39                         161   
2      14.840000       45.95                   39                         161   
3      14.287500       45.95                   39                         161   
4      15.100000       45.95                   39                         161   

   product_photos_qty  ...  comp_1  ps1        fp1      comp_2  

In [2]:
if not data.empty:
    # Drop rows with missing values for simplicity
    data = data.dropna()

    # Define features (X) and target (y) based on the dataset's columns
    # 'total_price' is what we want to predict
    target = 'total_price'

    # Features are all numeric columns except the target and identifiers
    features = data.select_dtypes(include=np.number).columns.tolist()

    # Remove the target variable and any ID-like columns from the feature list
    if target in features:
        features.remove(target)
    if 'product_id' in features:
        features.remove('product_id') # Assuming 'product_id' is just an identifier

    X = data[features]
    y = data[target]

    print("\nFeatures selected for modeling:")
    print(X.columns.tolist())
    print("\nTarget variable:")
    print(target)



Features selected for modeling:
['qty', 'freight_price', 'unit_price', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_score', 'customers', 'weekday', 'weekend', 'holiday', 'month', 'year', 's', 'volume', 'comp_1', 'ps1', 'fp1', 'comp_2', 'ps2', 'fp2', 'comp_3', 'ps3', 'fp3', 'lag_price']

Target variable:
total_price


In [3]:
if not data.empty:
    # Scale the features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Select the top 5 most relevant features
    # The number 'k' can be tuned
    selector = SelectKBest(score_func=f_regression, k=min(5, len(features)))
    X_selected = selector.fit_transform(X_scaled, y)

    # Get the names of the selected features
    selected_feature_names = X.columns[selector.get_support()]
    print("\nTop 5 features selected by algorithm:")
    print(selected_feature_names.tolist())

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)
    print(f"\nData split into training and testing sets.")
    print(f"Training set shape: {X_train.shape}, Test set shape: {X_test.shape}")



Top 5 features selected by algorithm:
['qty', 'unit_price', 'customers', 's', 'lag_price']

Data split into training and testing sets.
Training set shape: (540, 5), Test set shape: (136, 5)


In [4]:
if not data.empty:
    # Define the models to be trained
    models = {
        'LinearRegression': LinearRegression(),
        'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
        'GradientBoosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
    }

    # Train and evaluate each model
    results = []
    print("\n--- Model Performance ---")
    for name, model in models.items():
        model.fit(X_train, y_train)
        preds = model.predict(X_test)

        mae = mean_absolute_error(y_test, preds)
        r2 = r2_score(y_test, preds)

        results.append({'Model': name, 'MAE': mae, 'R2 Score': r2})
        print(f"{name}: MAE = {mae:.4f}, R2 Score = {r2:.4f}")

    # Create a DataFrame for easy comparison
    results_df = pd.DataFrame(results)



--- Model Performance ---
LinearRegression: MAE = 411.1520, R2 Score = 0.8335
RandomForest: MAE = 103.1259, R2 Score = 0.9490
GradientBoosting: MAE = 120.3432, R2 Score = 0.9612


In [5]:
if not data.empty:
    # Find the best model based on R2 Score
    best_model_info = results_df.loc[results_df['R2 Score'].idxmax()]
    best_model_name = best_model_info['Model']
    best_model = models[best_model_name]

    print("\n--- Best Model ---")
    print(f"The best performing model is: {best_model_name} with an R2 Score of {best_model_info['R2 Score']:.4f}")

    # Save the best model to a file
    joblib.dump(best_model, 'best_model.pkl')
    print(f"\nBest model ('{best_model_name}') has been saved as 'best_model.pkl'")

    # Save the performance comparison to a CSV file
    results_df.to_csv('model_performance_comparison.csv', index=False)
    print("Model performance comparison saved to 'model_performance_comparison.csv'")



--- Best Model ---
The best performing model is: GradientBoosting with an R2 Score of 0.9612

Best model ('GradientBoosting') has been saved as 'best_model.pkl'
Model performance comparison saved to 'model_performance_comparison.csv'
